In [6]:
import numpy as np
import pandas as pd
import os
import cv2
import dlib
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

data_dict = dict()
test_data = dict()
test_labels = dict()
#Read Training data
data = pd.read_csv('datasets/train.csv')
data_test = pd.read_csv('datasets/test.csv')

In [11]:
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [12]:
emotions = data['Emotion'].values
pixels = data['Pixels'].values
pixels_test = pixels
print(len(pixels)/2)


2089.0


In [13]:
pixels.shape[0]

4178

In [14]:
# Data Extration using Training set
images = []
target = []
landmarks = []
for i in range(int(len(pixels)/2)):
    pixels_formatted = np.fromstring(pixels[i],dtype=int, sep=" ").reshape((48,48))
    images.append(pixels_formatted)
    target.append(int(emotions[i]))
    cv2.imwrite('tempu.jpg',pixels_formatted)
    image= cv2.imread('tempu.jpg')
    face_rect= [dlib.rectangle(left=1, top=1, right=47, bottom = 47)]
    face_landmarks = np.matrix([[p.x, p.y] for p in predictor(image, face_rect[0]).parts()])
    landmarks.append(face_landmarks)

In [15]:
X_train = np.array(images)
Y_train = np.array(target)
print(Y_train[:10])
print(X_train.shape)
X_train = X_train.reshape(X_train.shape[0],48,48,1)
Y_train = np_utils.to_categorical(Y_train,7)
print(Y_train.shape)


[3 6 4 6 2 6 3 1 3 1]
(2089, 48, 48)
(2089, 7)


In [16]:
#Data Extraction using Test set
images_test = []
landmarks_test = []
target_test = []
for i in range(int(len(pixels_test)/2)+1,len(pixels_test)):
    pixels_formatted = np.fromstring(pixels_test[i],dtype=int, sep=" ").reshape((48,48))
    images_test.append(pixels_formatted)
    target_test.append(int(emotions[i]))
    cv2.imwrite('temp.jpg',pixels_formatted)
    image= cv2.imread('temp.jpg')
    face_rect= [dlib.rectangle(left=1, top=1, right=47, bottom = 47)]
    face_landmarks_test = np.matrix([[p.x, p.y] for p in predictor(image, face_rect[0]).parts()])
    landmarks_test.append(face_landmarks_test)

In [17]:
X_test = np.array(images_test)
Y_test = np.array(target_test)
print(Y_test[:10])
print(X_test.shape)
X_test = X_test.reshape(X_test.shape[0],48,48,1)
Y_test = np_utils.to_categorical(Y_test,7)
print(Y_test.shape)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


[3 5 5 6 5 1 4 0 3 6]
(2088, 48, 48)
(2088, 7)


In [18]:
np.save('../saved/images.npy',images)
np.save('../saved/targets.npy',target)
np.save('../saved/landmarks.npy',landmarks)
print(len(target_test))

FileNotFoundError: [Errno 2] No such file or directory: '../saved/images.npy'

In [283]:
model = Sequential()
 
model.add(Convolution2D(48, (4, 4), activation='relu', input_shape=(48,48,1)))
model.add(Convolution2D(16, (7, 7), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
 
model.add(Flatten())
model.add(Dense(144, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))
 

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 

model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1)
 
model.save('blah.h5')
score = model.evaluate(X_test, Y_test, verbose=1)

Epoch 1/10
2089/2089 [==============================] - 62s - loss: 1.9401 - acc: 0.2336    
Epoch 2/10
2089/2089 [==============================] - 57s - loss: 1.8634 - acc: 0.2901    
Epoch 3/10
2089/2089 [==============================] - 52s - loss: 1.8140 - acc: 0.3140    
Epoch 4/10
2089/2089 [==============================] - 53s - loss: 1.6540 - acc: 0.4031    
Epoch 5/10
2089/2089 [==============================] - 53s - loss: 1.3852 - acc: 0.5074    
Epoch 6/10
2089/2089 [==============================] - 54s - loss: 1.2370 - acc: 0.5711    
Epoch 7/10
2089/2089 [==============================] - 54s - loss: 1.0986 - acc: 0.6213    
Epoch 8/10
2089/2089 [==============================] - 53s - loss: 1.0077 - acc: 0.6477    
Epoch 9/10
2089/2089 [==============================] - 56s - loss: 0.9445 - acc: 0.6692    
Epoch 10/10
2088/2088 [==============================] - 20s    


In [274]:
score

[0.87753359122751318, 0.70210727969348663]

In [223]:
landmarks = np.array(landmarks)
target = np.array(target)
target_test = np.array(target_test)
landmarks_test = np.array(landmarks_test)
data_dict['X'] = np.array([x.flatten() for x in landmarks])
data_dict['Y'] = np.array(target)

test_data['X'] = np.array([x.flatten() for x in landmarks_test])
test_data['Y'] = np.array(target_test)

In [224]:
import time
import os
import _pickle as cPickle
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score 

In [225]:
#Training
model = AdaBoostClassifier(n_estimators = 70,learning_rate = 0.7, algorithm = 'SAMME.R')

In [226]:
start_time = time.time()
model.fit(data_dict['X'],data_dict['Y'])
training_time = time.time()-start_time
print(training_time)


0.6492621898651123


In [227]:
predicted_emotion = model.predict(test_data['X'])

In [228]:
print(accuracy_score(predicted_emotion,test_data['Y']))

0.585727969349


In [216]:
len(test_data['Y'])

2088

In [217]:
from pandas_ml import ConfusionMatrix as cm

In [218]:
print(cm(test_data['Y'],predicted_emotion))

Predicted    0    1    2    3    4    5    6  __all__
Actual                                               
0          134   25    5    4    9    3   35      215
1           38  113    4   15   23    2   19      214
2            8   16   39   18   17   44   79      221
3            1   19    8  325    0    1   23      377
4           22   12    5    5   42   14  127      227
5            8    3   40    6    5  115   44      221
6           29   17   20   10   49   33  455      613
__all__    240  205  121  383  145  212  782     2088
